In [1]:
from utility.readCsvFile import openCSVFile, readAllCSVFile, readRedTeam
import numpy as np

In [2]:
#mapnames to numbers
#mapUserComputerList[number] = name
mapUserComputerList = []

In [3]:
#test
data,_ = openCSVFile("data/authpartitionaa", mapUserComputerList)
#print(data)

In [4]:
#test on one file
np.array(data)
print(np.shape(data))

(3000, 3)


In [5]:
#read all files
fileNamePrefix = "data/authpartition"
data, mapList = readAllCSVFile(fileNamePrefix)


In [6]:
#check data shape
nparrayData = np.array(data)
print(np.shape(nparrayData))

(2028000, 3)


In [7]:
#separate X,y

X = nparrayData[:,[0,1]]
y = nparrayData[:,[2]]

print(np.shape(X))
print(np.shape(y))

(2028000, 2)
(2028000, 1)


In [8]:
#MLP
from sklearn.neural_network import MLPClassifier


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=110041455, shuffle=False)

In [10]:
#mlpClassifier = MLPClassifier()
#mlpClassifier = mlpClassifier.fit(X_train,y_train.ravel())

In [13]:
#set params
#use rule of thumb
#from
#https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw
#Nh = Ns / (a (Ni + No)) = 1800000/(2to 10(2+1)) = 1800000/(3*(2 to 10))
#Nh = 600000/2 to 600000/10 = 300000 to 60000
hiddenLayerList = []
for hiddenLayerCounter in np.arange(60000, 300001, 60000):
    hiddenLayerList.append((hiddenLayerCounter,))
    hiddenLayerList.append((hiddenLayerCounter/1000, hiddenLayerCounter/1000,hiddenLayerCounter/1000))
params = {"hidden_layer_sizes": hiddenLayerList,
         "activation": ['identity', 'logistic', 'tanh', 'relu'],
         "learning_rate": ['constant', 'invscaling', 'adaptive'],
          "solver": ['sgd', 'adam'],
         "alpha": [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05]}

In [16]:
from sklearn.model_selection import GridSearchCV
mlpClassifier = MLPClassifier(shuffle=False, n_jobs=-1)
grid = GridSearchCV(mlpClassifier, params)

In [ ]:
import time
startTime = time.time()
grid.fit(X_train,y_train.ravel())
endTime = time.time()

In [ ]:
print("Time taken: " + str(endTime - startTime))
searchScore = grid.score(X_test, y_test.ravel())
print("test score: "+str(searchScore))
print("Best param: "+grid.best_params_)

In [11]:
import pickle

In [12]:
folder = "classifier/"
classifierFileName = "MLPDefault.sav"
pickle.dump(mlpClassifier, open(folder+classifierFileName, "wb"))

In [13]:
"""
Load_mlpClassifier = pickle.load(open(folder+classifierFileName, "rb"))
"""

'\nLoad_mlpClassifier = pickle.load(open(folder+classifierFileName, "rb"))\n'

In [14]:
testFile = "data/redteam.txt"
testdata, mapList = readRedTeam(testFile, mapList)

In [15]:
testdata = np.array(testdata)

In [16]:
#separate testX,testy

testX = testdata[:,:2]
testy = testdata[:,[2]]

print(np.shape(testX))
print(np.shape(testy))

(749, 2)
(749, 1)


In [17]:
result = mlpClassifier.score(testX, testy)
print(result)

0.49132176234979974


In [18]:
#test red team
correct = 0
incorrect = 0

for t in testX:
    result = mlpClassifier.predict([t])
    #print(result)
    #threshold 0.5
    if result>0.5:
        correct = correct + 1
    else:
        incorrect = incorrect + 1

In [19]:
print("correct: "+str(correct)+" incorrect: "+str(incorrect))

correct: 368 incorrect: 381


In [20]:
print([X_test[0]])
print(y_test[0][0])

[array([24, 24])]
0


In [21]:
#test train data
correct = 0
falsepositive = 0
incorrect = 0
counter = 0
for t in X_test:
    result = mlpClassifier.predict([t])
    #print(result)
    #threshold 0.5
    if result>0.5 and (not y_test[counter] == 0):
        correct = correct + 1
    elif result < 0.5 and y_test[counter] == 0:
        falsepositive = falsepositive + 1
        correct = correct + 1
    else:
        incorrect = incorrect + 1
    counter = counter + 1

In [22]:
print("correct: "+str(correct)+" falsepositive: "+str(falsepositive)+" incorrect: "+str(incorrect))

correct: 201484 falsepositive: 201484 incorrect: 1316
